In [81]:
import pandas as pd
import pickle
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

In [97]:
train_df = pd.read_parquet('yellow_tripdata_2022-01.parquet')
test_df = pd.read_parquet('yellow_tripdata_2022-02.parquet')

train_df.head()

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,airport_fee
0,1,2022-01-01 00:35:40,2022-01-01 00:53:29,2.0,3.80,1.0,N,142,236,1,14.5,3.0,0.5,3.65,0.0,0.3,21.95,2.5,0.0
1,1,2022-01-01 00:33:43,2022-01-01 00:42:07,1.0,2.10,1.0,N,236,42,1,8.0,0.5,0.5,4.00,0.0,0.3,13.30,0.0,0.0
2,2,2022-01-01 00:53:21,2022-01-01 01:02:19,1.0,0.97,1.0,N,166,166,1,7.5,0.5,0.5,1.76,0.0,0.3,10.56,0.0,0.0
3,2,2022-01-01 00:25:21,2022-01-01 00:35:23,1.0,1.09,1.0,N,114,68,2,8.0,0.5,0.5,0.00,0.0,0.3,11.80,2.5,0.0
4,2,2022-01-01 00:36:48,2022-01-01 01:14:20,1.0,4.30,1.0,N,68,163,1,23.5,0.5,0.5,3.00,0.0,0.3,30.30,2.5,0.0


In [83]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2463931 entries, 0 to 2463930
Data columns (total 19 columns):
 #   Column                 Dtype         
---  ------                 -----         
 0   VendorID               int64         
 1   tpep_pickup_datetime   datetime64[ns]
 2   tpep_dropoff_datetime  datetime64[ns]
 3   passenger_count        float64       
 4   trip_distance          float64       
 5   RatecodeID             float64       
 6   store_and_fwd_flag     object        
 7   PULocationID           int64         
 8   DOLocationID           int64         
 9   payment_type           int64         
 10  fare_amount            float64       
 11  extra                  float64       
 12  mta_tax                float64       
 13  tip_amount             float64       
 14  tolls_amount           float64       
 15  improvement_surcharge  float64       
 16  total_amount           float64       
 17  congestion_surcharge   float64       
 18  airport_fee           

In [89]:
def preprocess_df(df):
    
    df["duration"] = df["tpep_dropoff_datetime"] - df['tpep_pickup_datetime']
    #convert duration to minutes
    df["duration"] = df["duration"].apply(lambda dt: dt.total_seconds()/60)
    # Dropping outliers
    filtered_df = df.query("duration >=1 & duration <= 60")
    #convert columns to string formats
    categorical = ['PULocationID', 'DOLocationID']
    filtered_df[categorical] = filtered_df[categorical].astype(str)

    return df, filtered_df


In [98]:
original_df, processed_train_df = preprocess_df(train_df)
_, processed_test_df = preprocess_df(test_df)

C:\Users\User\AppData\Local\Temp\ipykernel_3728\3691254614.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_df.loc[:, categorical] = filtered_df[categorical].astype(str)
C:\Users\User\AppData\Local\Temp\ipykernel_3728\3691254614.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_df.loc[:, categorical] = filtered_df[categorical].astype(str)


Standard Deviation

In [99]:
print(f"Standard deviation is: {original_df.duration.std():.2f}")

Standard deviation is: 46.45


In [100]:
print(f"{(len(processed_train_df)/len(train_df))*100:.2f} % records left")

98.28 % records left


One hot encoding

In [102]:
categorical = ['PULocationID', 'DOLocationID']

train_dicts = processed_train_df[categorical].to_dict(orient="records")
val_dicts = processed_test_df[categorical].to_dict(orient="records")

dv = DictVectorizer()

X_train = dv.fit_transform(train_dicts)
X_val = dv.transform(val_dicts)

Dimentionality

In [108]:
X_train.shape[1]

515

Model Training

In [106]:
y_train = processed_train_df["duration"].values

lr = LinearRegression()
lr.fit(X_train, y_train)

pred = lr.predict(X_train)

rmse = mean_squared_error(y_train, pred, squared=False)

print(f"the rmse is: {rmse:.2f}")

the rmse is: 6.99


In [105]:
y_val = processed_test_df["duration"].values

pred = lr.predict(X_val)

rmse = mean_squared_error(y_val, pred, squared=False)

print(f"the rmse on test data is: {rmse:.2f}")

the rmse on test data is: 7.79
